In [1]:
import sys


sys.version

'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:41:13) [MSC v.1900 64 bit (AMD64)]'

In [50]:
import keras
import os
import cv2
import random
import warnings
import numpy as np
import sys
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

%matplotlib inline
from keras.preprocessing import image

from keras.models import Model
from keras import layers
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import SeparableConv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape

keras.__version__

'2.0.8'

In [3]:
xcep_model = keras.applications.xception.Xception(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [4]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [23]:
def get_test_image(path):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


def image_preprocess_and_predict(path, model):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)
    preds = model.predict(x)
    print(np.argmax(preds))
    print('Predicted:', decode_predictions(preds, 1))

In [13]:
path = 'D:/DataSet/dogs_vs_cats/demo/'

filenames = '''
cat.375.jpg
cat.376.jpg
cat.377.jpg
dog.8120.jpg
dog.8121.jpg
dog.8122.jpg
'''

from sein import SeinString as ss
for name in ss(filenames).datalines():
    image_preprocess_and_predict(path + name)

Input image shape: (1, 299, 299, 3)
463
Predicted: [[('n02909870', 'bucket', 0.5654307)]]
Input image shape: (1, 299, 299, 3)
463
Predicted: [[('n02909870', 'bucket', 0.5654307)]]
Input image shape: (1, 299, 299, 3)
283
Predicted: [[('n02123394', 'Persian_cat', 0.43360224)]]
Input image shape: (1, 299, 299, 3)
208
Predicted: [[('n02099712', 'Labrador_retriever', 0.4385938)]]
Input image shape: (1, 299, 299, 3)
187
Predicted: [[('n02094433', 'Yorkshire_terrier', 0.33378917)]]
Input image shape: (1, 299, 299, 3)
248
Predicted: [[('n02109961', 'Eskimo_dog', 0.31434378)]]


In [35]:
xcmodel = keras.applications.xception.Xception(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=2)

In [44]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'
xcmodel.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
xcmodel

In [27]:
for name in ss(filenames).datalines():
    test_image = get_test_image(path + name)
    preds = xcep_model2.predict(test_image)
    print(np.argmax(preds))
    print(preds)


0
[[ 0.50000364  0.49999642]]
0
[[ 0.50000364  0.49999642]]
0
[[ 0.50000882  0.49999118]]
0
[[ 0.50000787  0.49999216]]
0
[[ 0.50000501  0.49999499]]
0
[[ 0.50000334  0.49999663]]


In [51]:
TRAIN_DIR = 'D:/DataSet/dogs_vs_cats/train/'
TEST_DIR = 'D:/DataSet/dogs_vs_cats/test/'

ROWS = 299
COLS = 299
CHANNELS = 3

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

TRAIN_SAMPLE = 1000 and 100
TEST_SAMPLE = 50
# slice datasets for memory efficiency on Kaggle Kernels, delete if using full dataset
train_images = train_dogs[:TRAIN_SAMPLE] + train_cats[:TRAIN_SAMPLE]
random.shuffle(train_images)
test_images =  test_images[:TEST_SAMPLE]

def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS, CHANNELS, ), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        
        data[i] = image
        # data[i] = image.T 
        # print(img.shape, img.T.shape)
        # => (299, 299, 3) (3, 299, 299)
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
    
    return data

In [52]:
train = prep_data(train_images)
test = prep_data(test_images)

print("Train shape: {}".format(train.shape))
print("Test shape: {}".format(test.shape))

Processed 0 of 200
Processed 0 of 50
Train shape: (200, 299, 299, 3)
Test shape: (50, 299, 299, 3)


In [53]:
labels = []
# print(train_images[:10])
for i in train_images:
    if 'dog.' in i:      # folder name contain D:/DataSet/dogs_vs_cats/
        labels.append((1, 0))
    else:
        labels.append((0, 1))
# print(labels[:10])
# sns.countplot(labels)
# plt.title('Cats and Dogs')
# plt.show()

In [62]:
import time
epochs = 10
batch_size = 10

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.start_time = time.time()
        
    def on_batch_end(self, batch_idx, logs={}):
        info = '\r current epoch {}/{}  elapsed {:.2f}'
        info = info.format(batch_size*(batch_idx+1), TRAIN_SAMPLE*2, time.time() - self.start_time)
        print(info, end='', flush=True)
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        print('loss', logs.get('loss'), 'val_loss', logs.get('val_loss'))

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')        
        
def run_catdog():
    
    history = LossHistory()
    xcmodel.fit(train, labels, batch_size=batch_size, epochs=epochs,
                validation_split=0.25, verbose=0, shuffle=True, callbacks=[history, early_stopping])
    

    predictions = xcmodel.predict(test, verbose=0)
    return predictions, history

predictions, history = run_catdog()



 current epoch 150/200  elapse 467.30loss 0.574591871103 val_loss 0.694159674644
 current epoch 150/200  elapse 980.77loss 0.461858791113 val_loss 0.694163739681
 current epoch 150/200  elapse 1502.58loss 0.493965038657 val_loss 0.694308996201
 current epoch 150/200  elapse 2044.95loss 0.314564647277 val_loss 0.694744849205
 current epoch 150/200  elapse 2569.71loss 0.232443934182 val_loss 0.696743631363
Epoch 00004: early stopping


NameError: name 'model' is not defined

In [63]:
predictions = xcmodel.predict(test, verbose=0)
predictions

array([[ 0.48687351,  0.51312655],
       [ 0.48689407,  0.51310593],
       [ 0.48691782,  0.51308215],
       [ 0.4869298 ,  0.51307017],
       [ 0.48686993,  0.51313013],
       [ 0.4867498 ,  0.51325023],
       [ 0.48720551,  0.51279449],
       [ 0.48686895,  0.51313114],
       [ 0.48667136,  0.51332867],
       [ 0.48705131,  0.51294863],
       [ 0.48722267,  0.51277733],
       [ 0.48690814,  0.5130918 ],
       [ 0.48682782,  0.51317227],
       [ 0.48660505,  0.51339495],
       [ 0.48675805,  0.51324189],
       [ 0.48732266,  0.51267731],
       [ 0.48688251,  0.51311749],
       [ 0.48690051,  0.51309949],
       [ 0.4870477 ,  0.51295233],
       [ 0.48670515,  0.51329488],
       [ 0.48716035,  0.51283967],
       [ 0.48679852,  0.51320142],
       [ 0.48639536,  0.51360458],
       [ 0.48678747,  0.51321256],
       [ 0.48719829,  0.51280177],
       [ 0.48698324,  0.51301682],
       [ 0.48684782,  0.51315218],
       [ 0.48715889,  0.51284111],
       [ 0.48716232,

In [ ]:
loss = history.losses
val_loss = history.val_losses

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('VGG-16 Loss Trend')
plt.plot(loss, 'blue', label='Training Loss')
plt.plot(val_loss, 'green', label='Validation Loss')
plt.xticks(range(0, epochs)[0::2])
plt.legend()
plt.show()


for i in range(0,10):
    if predictions[i, 0] >= 0.5: 
        print('I am {:.2%} sure this is a Dog'.format(predictions[i][0]))
    else: 
        print('I am {:.2%} sure this is a Cat'.format(1-predictions[i][0]))
        
    plt.imshow(test[i].T)
    plt.show(figsize=(2,2))